In [1]:
import os
import pandas as pd
import re

class Expeds(object):

    def get_data(self):
        """
        This function get the data from the csv file and return a DataFrame.
        """
        root_dir = os.path.abspath('')
        xls_path = os.path.join(root_dir, 'data')
        file_names = [f for f in os.listdir(xls_path) if f.endswith('.xls')]

        def key_from_file_name(f):
            if f[-4:] == '.xls':
                return f[:-4]
        data = {}
        for f in file_names:
            data[key_from_file_name(f)] = pd.read_excel(os.path.join(xls_path, f))

        data = data['members']
        return data

    def clean_expeds(self):
        df = get_data()
        df = df.copy()

        
        ## defining some definitions for the cleaning 
        def numbers(x):
            if type(x) == str:
                return int(re.findall(r"\d*", x)[0])
            return x

        def true(x):
            if x == 0:
                return False
            return True

        def no_sponsor(x):
            if x == float('nan'): 
                return False
            return x

        def no_route(x):
            if x == float('nan'): 
                return 'other' 
            return x


        ## Applying functions to the specific columns    
        df['ascent1'].fillna(0, inplace=True)
        df['ascent1'] = df['ascent1'].map(numbers)

        df['route1'].fillna('other', inplace = True)
        df['route2'].fillna('other', inplace = True)

        df['achievment'].fillna(0, inplace=True)
        df['achievment'] = df['achievment'].map(true)

        df['sponsor'] = df['sponsor'].map(no_sponsor)
        df['agency'] = df['agency'].map(no_sponsor)

        df['termnote'] = df['termnote'].map({
            0 : 'Unknown',
            1 : 'Success_main',
            2 : 'Success_sub',
            3 : 'Success_claim',
            4 : 'Bad_weather',
            5 : 'Bad_conditions',
            6 : 'Accident',
            7 : 'Illness',
            8 : 'Lack_sse',
            9 : 'Lack_time',
            10 : 'lack_of_motivation',
            11 : 'no_reach_camp',
            12 : 'no_attempt_climb',
            13 : 'Attempt_rumored',
            14 : "Other"})


        ## dropping 
        df.drop(columns=['route3', 'route4', 
        'ascent3', 
        'ascent4', 
        'ascent3',
        'approach', 
        'primref', 
        'primid',
        'chksum', 
        'success3',
        'success4'],  inplace = True)
        
        ## Filling with zeros because its makes sense
        df['summit_time'].fillna(0, inplace=True)
        df['other_smts'].fillna(0, inplace=True)

        df['season'] = df['season'].map({
                        0 : 'Unknown',
                        1 : 'Spring',
                        2 : 'Summer',
                        3 : 'Autumn',
                        4 : 'Winter'})

        df['host'] =  df['host'].map({
            0 : 'Unknown',
            1 : 'Nepal',
            2 : 'China',
            3 : 'India'})

        return df

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import datetime
import os
import pandas as pd

class Members:

    def get_data(self):
        """
        This function get the data from the csv file and return a DataFrame.
        """
        root_dir = os.path.abspath('')
        xls_path = os.path.join(root_dir, 'data')
        file_names = [f for f in os.listdir(xls_path) if f.endswith('.xls')]

        def key_from_file_name(f):
            if f[-4:] == '.xls':
                return f[:-4]
        data = {}
        for f in file_names:
            data[key_from_file_name(f)] = pd.read_excel(os.path.join(xls_path, f))

        data = data['members']
        return data

    def clean_data(self, df):
        """
        This function takes a DataFrame and returns a clean DataFrame.
        """
        data = df.copy()

        col_to_drop = ['first_name',
               'last_name',
               'age',
               'birthdate',
               'route2',
               'route3',
               'ascent2',
               'ascent3',
               'summit_time1',
               'summit_time2',
               'summit_time3',
               'summit_date2',
               'summit_date3',
               'o2_note',
               'death_date',
               'death_time',
               'hcn',
               'mchksum']

        data.drop(columns= col_to_drop, inplace = True)

        data['season'] = data['season'].map({
                        0 : 'Unknown',
                        1 : 'Spring',
                        2 : 'Summer',
                        3 : 'Autumn',
                        4 : 'Winter'})

        data['death_type'] = data['death_type'].map({
                        0 : 'Unspecified',
                        1 : 'AMS',
                        2 : 'Exhaustion',
                        3 : 'Exposure / frostbite',
                        4 : 'Fall',
                        5 : 'Crevasse',
                        6 : 'Icefall collapse',
                        7 : 'Avalanche',
                        8 : 'Falling rock / ice',
                        9 : 'Disappearance',
                        10 : 'Illness (non-AMS)',
                        11 : 'Other',
                        12 : 'Unknown'})

        data['death_class'] = data['death_class'].map({
                        0 : 'Unspecified',
                        1 : 'Death enroute BC',
                        2 : 'Death at BC / ABC',
                        3 : 'Route preparation',
                        4 : 'Ascending in summit bid',
                        5 : 'Descending from summit bid',
                        6 : 'Expedition evacuation',
                        7 : 'Other / Unknown'})

        data['summit_bid'] = data['summit_bid'].map({
                        0 : 'Unspecified',
                        1 : 'No summit bid',
                        2 : 'Aborted below high camp',
                        3 : 'Aborted at high camp',
                        4 : 'Aborted above high camp',
                        5 : 'Successful summit bid'})

        data.rename(columns={'summitter' : 'summit_term'}, inplace=True)

        data['summit_term'] = data['summit_term'].map({
                        0 : 'Unspecified',
                        1 : 'Success',
                        2 : 'Success subpeak',
                        3 : 'Bad weather',
                        4 : 'Bad conditions',
                        5 : 'Accident',
                        6 : 'AMS',
                        7 : 'Exhaustion',
                        8 : 'Frostbite',
                        9 : 'Illnesses',
                        10 : 'Logisitic',
                        11 : 'O2 system failure',
                        12 : 'Route difficulty',
                        13 : 'Too late/slow',
                        14 : 'Assisting',
                        15 : 'Route/camp/rope preparation',
                        16 : 'No time left',
                        17 : 'No climbing',
                        18 : 'Other',
                        19 : 'Unknown'})

        data['summit_date1'] = pd.to_datetime(data.summit_date1, errors = 'coerce')

        # Combine "age_transformer" and "One hot encoder" into a single preprocessor
        preprocessor = ColumnTransformer([
            ('median', SimpleImputer(strategy='median'), ['yob']),
            ('frequency', SimpleImputer(strategy='most_frequent'), ['citizenship', 'status']),
            ('ohe', OneHotEncoder(drop= 'first'), ['sex'])])

        # Pass the combined preprocessor into a Pipeline as a single step
        final_pipe = Pipeline([('preprocessing', preprocessor)])

        data_trans = final_pipe.fit_transform(data)

        data[['yob', 'citizenship', 'status', 'sex']] = data_trans

        data.rename(columns={'sex' : 'sex_M'}, inplace=True)

        data.occupation.fillna('Unknown', inplace = True)
        data.residence.fillna('Unknown', inplace = True)

        now_year = datetime.datetime.now().year
        data['age'] = now_year - data['yob']

        return data

In [3]:
df =  Members().get_data()

In [4]:
data = Members().clean_data(df)

In [9]:
data.to_csv("C:\\Users\\jerem\\code\\JeremyDel\\himalaya-project\\himalaya-project\\data\\clean")

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\jerem\\code'

In [24]:
from members import Members
member = Members().get_data()
member = Members().clean_data(member)


In [25]:
from expeds import Expeds
exped = Expeds().get_data()
exped = Expeds().clean_data(exped)

In [31]:
df = exped.merge(member, on='exp_id', how = 'right')

In [32]:
df.status

0            Leader
1        BC Manager
2           Climber
3            Leader
4           Climber
            ...    
16378       Climber
16379    H-A Worker
16380        Leader
16381       Climber
16382        Leader
Name: status, Length: 16383, dtype: object

In [33]:
def worker(x):
    if x == 'H-A Worker':
        return 1
    return 0
df.status= df.status.map(worker)

In [34]:
import numpy as np
workers = df.groupby('exp_id', as_index=False).agg({'tot_members': 'count', 'status': np.sum })

In [35]:
workers['ratio_workers'] = workers.status / workers.tot_members

In [1]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from members import Members
from weather import Weather
from expeds import Expeds
import joblib
import os
from pickle import dump, load

class HimalXGB():

    def train_model(self):
        """
        This function get the data from csv files and return a trained model.
        It also save the model under the name XGB_model.joblib
        """

        # Get Data
        member = Members().get_data()
        member = Members().clean_data(member)

        weather = Weather().get_data()
        weather = Weather().clean_data(weather)

        exped = Expeds().get_data()
        exped = Expeds().clean_data(exped)

        # Drop columns
        mem_to_drop = ['memb_id','year','unique_id','peak_id','residence','occupation',
        'summit_claimed','summit_disputed','highpt','high_point','death','death_type',
        'death_height','death_class','summit_term','summit_date1', 'summit_bid',
        'citizenship','o2_climb','o2_descent','o2_sleep','o2_medical', 'o2_none',
        'yob', 'route1', 'ascent1', 'leader', 'deputy', 'bconly', 'nottobc', 'support',
        'hired', 'sherpa', 'tibetan']


        exp_to_drop = ['year','season','route1','route2','nation','leaders',
        'sponsor','success1','success2', 'ascent1','claimed','disputed',
        'countries','summit_time','term_date','term_note','high_point',
        'traverse','ski','parapente','o2_climb','o2_descent','o2_sleep',
        'o2_medical','o2_taken','o2_unkwn','o2_used','o2_none','other_smts',
        'campsites','accidents','achievment','agency','peak_name','primmem',
        'summiter_deaths','summit_members','summit_hired','hired_deaths']

        member.drop(columns= mem_to_drop, inplace=True)
        exped.drop(columns= exp_to_drop, inplace=True)

        exped['summit_date'] = pd.to_datetime(exped.summit_date, errors = 'coerce')
        exped['bc_date'] = pd.to_datetime(exped.bc_date , errors = 'coerce')
        exped['rope'] = np.where(exped['rope']>0, True, False)

        exped = exped.set_index('summit_date')
        weather = weather.set_index('date_time')

        # Feature Engineering (1/2)
        exped['sherpa_ratio'] = exped['tot_hired'] / exped['tot_members']
        exped['sherpa_ratio'] = np.where(exped['sherpa_ratio'] == np.inf, 0, exped['sherpa_ratio'])

        weather['pressure_past'] = weather['pressure'].rolling(window=3).mean()
        weather['pressure_futur'] = weather['pressure'].shift(-2).rolling(window=3).mean()
        weather['stability'] = weather['pressure_futur'] - weather['pressure_past']

        # Merge DataFrames
        df = exped.merge(weather, how='left', left_index=True, right_index=True)
        df = df.reset_index()
        df = df.rename(columns={'index' : 'summit_date'})
        df = df.merge(member, on='exp_id', how = 'right')
        df = df.dropna(subset=['summit_date', 'bc_date'])

        # Feature Engineering (2/2)
        df['cumul_snow'] = 0

        for index, row in df.iterrows():
            date1 = row['bc_date'].date()
            date2 = row['summit_date'].date()
            acc_snow = weather.loc[date1:date2, 'totalSnow_cm'].sum()
            df.loc[index, 'cumul_snow'] = acc_snow

        feature_to_drop = ['tempC', 'WindChillC', 'primrte', 'disabled','moonrise', 'moonset',
                           'sunrise', 'sunset', 'traverse', 'parapente', 'solo', 'ski', 'speed',
                           'summit_date', 'exp_id', 'bc_date', 'term_reason', 'tot_days',
                           'pressure_past', 'pressure_futur', 'uvIndex', 'o2_used']

        df.drop(columns= feature_to_drop, inplace=True)

        col_num = []
        col_bool =[]
        col_object =[]
        for col in df:
            if df[col].dtype == "float64":
                col_num.append(col)
            if df[col].dtype == "int64":
                col_num.append(col)
            if df[col].dtype == 'bool':
                col_bool.append(col)
            if df[col].dtype == 'object':
                col_object.append(col)

        col_bool.remove('summit_success')

        numeric_transformer = make_pipeline(SimpleImputer(), MinMaxScaler())

        categorical_transformer = make_pipeline(SimpleImputer(strategy='most_frequent'),
                                                OneHotEncoder(drop= 'first', handle_unknown='error'))
        feateng_blocks = [
                    ('num', numeric_transformer, col_num),
                    ('cat', categorical_transformer, col_object),
                ]
        features_encoder = ColumnTransformer(feateng_blocks, n_jobs=None, remainder="passthrough")

        pipeline = Pipeline([
                    ('features', features_encoder)])

        X = df.drop(columns=['summit_success'])
#         X = pipeline.fit_transform(X)

#         y = df.summit_success

#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state= 1)

#         # Model Traning
#         boost = XGBClassifier()
#         boost.fit(X_train, y_train)

#         # Save trained model
#         model_name = 'XGB_model.joblib'
#         joblib.dump(boost, model_name)

#         # Export pipeline as pickle file
#         with open("pipeline.pkl", "wb") as file:
#             dump(pipeline, file)

        return X

In [3]:
X_trains = HimalXGB().train_model()

In [4]:
X_trains

,peak_id,host,summit_days,camps,rope,tot_members,tot_hired,no_hired,comrte,stdrte,...,visibility,winddirDegree,windspeedKmph,date_season,stability,season,sex_M,status,age,cumul_snow
0,TKRG,Nepal,6,0,False,3,0,True,False,False,...,10.0,270.0,15.0,Winter,-4.0,Spring,1.0,leader,43.0,0.0
1,TKRG,Nepal,6,0,False,3,0,True,False,False,...,10.0,270.0,15.0,Winter,-4.0,Spring,0.0,bc member,39.0,0.0
2,TKRG,Nepal,6,0,False,3,0,True,False,False,...,10.0,270.0,15.0,Winter,-4.0,Spring,1.0,climber,37.0,0.0
6,AMAD,Nepal,16,0,False,2,0,True,True,False,...,10.0,265.0,17.0,Spring,4.0,Spring,1.0,leader,46.0,4.1
7,AMAD,Nepal,16,0,False,2,0,True,True,False,...,10.0,265.0,17.0,Spring,4.0,Spring,1.0,climber,39.0,4.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26493,TENR,Nepal,20,2,False,3,0,True,False,False,...,10.0,270.0,7.0,Autumn,0.0,Autumn,1.0,climber,28.0,6.6
26494,TENR,Nepal,20,2,False,3,0,True,False,False,...,10.0,270.0,7.0,Autumn,0.0,Autumn,1.0,climber,37.0,6.6
26502,TKPO,Nepal,7,4,False,3,0,True,False,False,...,10.0,269.0,6.0,Autumn,1.0,Autumn,1.0,leader,24.0,0.4
26503,TKPO,Nepal,7,4,False,3,0,True,False,False,...,10.0,269.0,6.0,Autumn,1.0,Autumn,1.0,media,42.0,0.4


In [5]:
weather = Weather().get_data()
weather = Weather().clean_data(weather)

In [7]:
water = pd.read_csv('data/clean/weather_clean.csv')

In [8]:
water

,Unnamed: 0,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,...,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,0,2010-01-01,-7,-18,0.0,8.7,2,100,2010-01-01 17:55:00,2010-01-01 07:09:00,...,-23,44,3,67,0.0,1023,-7,10,180,23
1,1,2010-01-02,-2,-17,0.0,8.7,2,85,2010-01-02 19:07:00,2010-01-02 08:01:00,...,-17,31,5,73,0.0,1023,-2,9,308,16
2,2,2010-01-03,-1,-13,0.0,8.7,2,77,2010-01-03 20:16:00,2010-01-03 08:45:00,...,-14,25,3,68,0.0,1019,-1,10,278,14
3,3,2010-01-04,-3,-16,0.0,8.7,2,70,2010-01-04 21:23:00,2010-01-04 09:26:00,...,-18,35,9,78,0.0,1015,-3,10,274,21
4,4,2010-01-05,-4,-21,0.0,8.7,2,63,2010-01-05 22:27:00,2010-01-05 10:02:00,...,-19,25,10,78,0.0,1017,-4,7,269,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,3953,2020-10-28,-8,-16,0.0,8.7,2,78,2020-10-28 15:49:00,2020-10-28 03:03:00,...,-17,13,0,29,0.0,1018,-8,10,222,7
3954,3954,2020-10-29,-8,-21,0.0,8.7,2,85,2020-10-29 16:18:00,2020-10-29 03:55:00,...,-18,22,0,28,0.0,1020,-8,10,226,11
3955,3955,2020-10-30,-7,-14,0.0,8.7,2,93,2020-10-30 16:48:00,2020-10-30 04:46:00,...,-17,19,0,32,0.0,1020,-7,10,224,9
3956,3956,2020-10-31,-8,-20,0.0,8.7,2,97,2020-10-31 17:19:00,2020-10-31 05:38:00,...,-18,18,0,41,0.0,1020,-8,10,220,9


In [9]:
weather

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,date_season
0,2010-01-01,-7,-18,0.0,8.7,2,100,2010-01-01 17:55:00,2010-01-01 07:09:00,2010-01-01 06:49:00,...,44,3,67,0.0,1023,-7,10,180,23,Winter
1,2010-01-02,-2,-17,0.0,8.7,2,85,2010-01-02 19:07:00,2010-01-02 08:01:00,2010-01-02 06:49:00,...,31,5,73,0.0,1023,-2,9,308,16,Winter
2,2010-01-03,-1,-13,0.0,8.7,2,77,2010-01-03 20:16:00,2010-01-03 08:45:00,2010-01-03 06:49:00,...,25,3,68,0.0,1019,-1,10,278,14,Winter
3,2010-01-04,-3,-16,0.0,8.7,2,70,2010-01-04 21:23:00,2010-01-04 09:26:00,2010-01-04 06:50:00,...,35,9,78,0.0,1015,-3,10,274,21,Winter
4,2010-01-05,-4,-21,0.0,8.7,2,63,2010-01-05 22:27:00,2010-01-05 10:02:00,2010-01-05 06:50:00,...,25,10,78,0.0,1017,-4,7,269,13,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,2020-10-28,-8,-16,0.0,8.7,2,78,2020-10-28 15:49:00,2020-10-28 03:03:00,2020-10-28 06:06:00,...,13,0,29,0.0,1018,-8,10,222,7,Autumn
3954,2020-10-29,-8,-21,0.0,8.7,2,85,2020-10-29 16:18:00,2020-10-29 03:55:00,2020-10-29 06:07:00,...,22,0,28,0.0,1020,-8,10,226,11,Autumn
3955,2020-10-30,-7,-14,0.0,8.7,2,93,2020-10-30 16:48:00,2020-10-30 04:46:00,2020-10-30 06:07:00,...,19,0,32,0.0,1020,-7,10,224,9,Autumn
3956,2020-10-31,-8,-20,0.0,8.7,2,97,2020-10-31 17:19:00,2020-10-31 05:38:00,2020-10-31 06:08:00,...,18,0,41,0.0,1020,-8,10,220,9,Autumn


In [10]:
water.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3958 entries, 0 to 3957
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         3958 non-null   int64  
 1   date_time          3958 non-null   object 
 2   maxtempC           3958 non-null   int64  
 3   mintempC           3958 non-null   int64  
 4   totalSnow_cm       3958 non-null   float64
 5   sunHour            3958 non-null   float64
 6   uvIndex            3958 non-null   int64  
 7   moon_illumination  3958 non-null   int64  
 8   moonrise           3958 non-null   object 
 9   moonset            3958 non-null   object 
 10  sunrise            3958 non-null   object 
 11  sunset             3958 non-null   object 
 12  DewPointC          3958 non-null   int64  
 13  FeelsLikeC         3958 non-null   int64  
 14  HeatIndexC         3958 non-null   int64  
 15  WindChillC         3958 non-null   int64  
 16  WindGustKmph       3958 

In [12]:
weather.to_csv('weather_clean.csv')